# VREF Filter

Objective: Design a Digital (PWM) to Analog filter with fast response time for setting voltage references in motor drives.




In [10]:
using Pkg
Pkg.add("ControlSystems")
Pkg.add("Plots")
Pkg.add("PlotlyBase")

  Resolving package versions...
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Project.toml`
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Manifest.toml`
  Resolving package versions...
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Project.toml`
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Manifest.toml`
  Resolving package versions...
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Project.toml`
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Manifest.toml`


In [11]:
using ControlSystems
using Plots
using PlotlyBase
plotly(); #nicer plots backend when running live
#gr(); # use GR for publish

Construct the transfer function of an RC low pass filter given by:

$$
\frac{1}{1+RCs}
$$

In [12]:
function lowpass_filter(R,C)
    s = tf("s")
    1/(1+R*C*s)
end

lowpass_filter (generic function with 1 method)

In [13]:
"""
Generate a PWM signal given a base clock and resolution (in bits).

clock::Int e.g. 24_000_000
resolution::Int e.g. 8 bit
duty_cycles::Vector{Int} e.g. [0, 128, 255, 64]
"""
function PWM(clock, resolution, values)

    period = resolution/clock
    bitperiod = period/(2^resolution)
    delays = DelayLtiSystem{Float64,Float64}[]
    
    for i in eachindex(values)
        offset = (i-1)*period
        push!(delays, delay(offset)) # initial
        push!(delays, -delay(offset+values[i]*bitperiod)) # turnoff
    end

    return sum(delays)
end

PWM

In [14]:
# Step function for LTI sim
function u0(out,t)
    if t > 0
        out[1] = 1 #Volts, but doesnt really matter so it is easier to see duty cycle
    else
        out[1] = 0
    end
    return
end

u0 (generic function with 1 method)

In [15]:
# 1.0b filter params

f1 = lowpass_filter(1e3,10e-9)

input = PWM(24_000_000, 8, fill(128, 500)) #,fill(64, 500),fill(255, 500), dims=1))

sys = ss(f1)*input

t = 0:5e-8:(input.Tau[end])


@time y, t, x = lsim(sys, u0, t)


 26.660447 seconds (179.63 M allocations: 4.346 GiB, 2.55% gc time)


([0.0; 0.0029712709293839416; … ; 0.5009525286848883; 0.5033694001111831], 0.0:5.0e-8:0.00016649999999999998, [0.0; 2.971270929383942e-8; … ; 5.009525286848883e-6; 5.033694001111832e-6])

In [16]:
plot(t, y)

<!DOCTYPE html>
 
 
 Plots.jl